# Реализовать разложение по сингулярным значениям (SVD) для ленточных матриц.
## создание произвольной ленточной матрицы.
На вход подаётся массив $n \times m$ - в таком виде хранятся ленточные матрицы.
$n$ - размер матрицы, $m$ - ширина ленты, нечётное число.
<img src='Matrix_storage-2.png'></img>

In [1]:
import numpy as np

In [2]:
#вместо tape назвать band
def random_tape(n, m, seed=1234):
    '''
    создаёт случайную ленту, на основе которой будет создана ленточная матрица
    n - длина ленты
    m - ширина ленты
    '''
    tape = np.zeros((n, m))
    l = (m - 1) // 2 #полуширина ленты
    for j in range(l, -1, -1):
        tape[j:, l - j] = np.random.rand(n - j)
    for j in range(1, l + 1):
        tape[:n - j, l + j] = np.random.rand(n - j)
    return tape
A=random_tape(5,5)
print(A)

[[0.         0.         0.68831554 0.46178128 0.30900665]
 [0.         0.28469414 0.05586811 0.28444679 0.70867623]
 [0.40915974 0.55580477 0.84156159 0.08123455 0.80015651]
 [0.47057799 0.67758015 0.56173123 0.96863533 0.        ]
 [0.5567152  0.0583857  0.92426124 0.         0.        ]]


In [3]:
def tape_matrix(tape):
    '''
    создаёт ленточную матрицу из ленты tape
    крайние элементы столбцов ленты должны содержать нули согласно рисунку выше
    '''
    n, m = tape.shape
    l = (m - 1) // 2
    matrix = np.zeros((n, n))
    
    i = 0
    for j in range(l, -1, -1):
        matrix[i, :i + l + 1] = tape[i, j:]
        i += 1
    start = i
    i = n - 1
    for j in range(l, 0, -1):
        matrix[i, i - l:] = tape[i, : -j]
        i -= 1
    stop = i
    for i in range(start, stop + 1):
        matrix[i, i - l : i + l + 1] = tape[i]
    return matrix

In [4]:
print(tape_matrix(A))
#A = tape_matrix(random_tape(5, 3))

[[0.68831554 0.46178128 0.30900665 0.         0.        ]
 [0.28469414 0.05586811 0.28444679 0.70867623 0.        ]
 [0.40915974 0.55580477 0.84156159 0.08123455 0.80015651]
 [0.         0.47057799 0.67758015 0.56173123 0.96863533]
 [0.         0.         0.5567152  0.0583857  0.92426124]]


<img src='one_way-2.png'></img>

In [5]:
A2 = A.T @ A
print(A2)

[[0.69878714 0.57877144 1.12322175 0.48905637 0.32739183]
 [0.57877144 0.85249344 0.91823083 0.78245896 0.64648677]
 [1.12322175 0.91823083 2.35492626 0.94621933 0.92566747]
 [0.48905637 0.78245896 0.94621933 1.23900538 0.40927452]
 [0.32739183 0.64648677 0.92566747 0.40927452 1.23795755]]


In [6]:
#спектральное разложение w - собственные значения, v - собственные вектора
w, v = np.linalg.eig(A2)
n = A2.shape[0]
#заполнения лямда собственными значениями
L = np.zeros((n, n))
for i in range(n):
    L[i, i] = w[i]
#проверка правильности разложения
np.allclose(A2, v @ L @ np.linalg.inv(v))

True

In [7]:
# находим сумму
Sigma = np.sqrt(L)

In [8]:
#находим матрицу u
u = A @ v @ np.linalg.inv(Sigma)

In [9]:
#проверка правильности
np.allclose(A, u @ Sigma @ np.linalg.inv(v))

True

In [10]:
#нахождение u и суммы встроенным методом
u_qr, Sigma_qr = np.linalg.qr(A @ v)

In [11]:
#проверка правильности
np.allclose(A, u_qr @ Sigma_qr @ np.linalg.inv(v))

True

In [12]:
#проверка правильности
np.allclose(A, u_qr @ Sigma_qr @ v.T)

True